In [1]:
import fastapi

In [2]:
router = fastapi.APIRouter()
router

In [3]:
import os
os.environ["CONFIG_PATH"] = '/export/scratch/lle/projects/hoaem/monorepo/common-libs/fastapi-utils/tests/data/configs'
import utils
config = utils.load_config(os.environ["CONFIG_PATH"])

In [24]:
import core
import pydantic as pdt
from typing import Any

class Model(pdt.BaseModel):
    name: str
    age: int = pdt.Field(default_factory=lambda: 32)
class CreateModelCommand(core.Command):
    name: str = pdt.Field(default='default_name')
    age: int = pdt.Field(default_factory=lambda: 32)

def create_model(command: CreateModelCommand):
    return command.name

def extract_schema_fields(model: pdt.BaseModel) -> dict[str, tuple[Any, Any, Any]]:
    fields_dict = {}
    for field_name, field_info in model.model_fields.items():
        # print(field_name)
        # print(field_info.annotation.__name__)
        # print(field_info.default_factory)
        # print(field_info.default)
        fields_dict[field_name] = (field_info.annotation, pdt.Field(default_factory=field_info.default_factory, default=field_info.default,))
    #     field_type = field_info.type_
    #     default_factory = field_info.default_factory
    #     default = field_info.default
    #     fields_dict[field_name] = (field_type, default_factory, default)
    return fields_dict

extract_schema_fields(CreateModelCommand)

{'name': (str,
  FieldInfo(annotation=NoneType, required=False, default='default_name')),
 'age': (int,
  FieldInfo(annotation=NoneType, required=False, default_factory=<lambda>))}

In [25]:
def build_command_(action: str, model_schema: pdt.BaseModel):
    print(action.capitalize())
    print(model_schema.__name__)
    print(extract_schema_fields(model_schema))
    Command = pdt.create_model(
        f"{action.capitalize()}{model_schema.__name__}Command",
        **extract_schema_fields(model_schema),
    )
    return Command

Command = build_command_('create', Model)

Create
Model
{'name': (<class 'str'>, FieldInfo(annotation=NoneType, required=True)), 'age': (<class 'int'>, FieldInfo(annotation=NoneType, required=False, default_factory=<lambda>))}


In [26]:
Command(name="1")

CreateModelCommand(name='1', age=32)

In [28]:
Command.__name__

'CreateModelCommand'

In [36]:
def snake_to_camel(snake_str: str) -> str:
    components = snake_str.split('_')
    return ''.join(x.title() for x in components)


In [44]:
from fastapi_utils.resources.crud import camel_to_snake

resource = camel_to_snake('CartItem')
class CartItemResponse(pdt.BaseModel):
    """"""
Response = CartItemResponse
code = f"""
def create_{resource}(command: {Command.__name__}) -> {Response.__name__}:
    bus = bootstrap.bootstrap()
    bus.handle(command)
    resource = view.fetch_model(
        model_cls=models.{snake_to_camel(resource)},
        creation_id=command._id,
    )
    """
from pprint import pprint as pp
pp(code)

('\n'
 'def create_cart_item(command: CreateModelCommand) -> CartItemResponse:\n'
 '    bus = bootstrap.bootstrap()\n'
 '    bus.handle(command)\n'
 '    resource = view.fetch_model(\n'
 '        model_cls=models.CartItem,\n'
 '        creation_id=command._id,\n'
 '    )\n'
 '    ')


In [50]:
def build_handler(code):
    local_ns = {}
    exec(code,
        globals(),
        local_ns,)
    return local_ns[f"create_{resource}"]
f = build_handler(code)
f(Command(name="1"))

NameError: name 'bootstrap' is not defined

In [46]:
router.add_api_route("/cart-items", create_cart_item, methods=["POST"])

NameError: name 'create_cart_item' is not defined

In [7]:
app = fastapi.FastAPI()
app.include_router(router)

In [8]:
from fastapi.testclient import TestClient

rest = TestClient(app)
response = rest.post("/models", json={"name": "test"})
response

<Response [200 OK]>

In [9]:
exec(
    """
def create_model(command: CreateModelCommand):
    return {"name": command.name}
    """,
    globals(), {},
)

In [12]:
create_model(CreateModelCommand(name="test"))

'test'